In [11]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [2]:
# Methods to load all the file as a floating point time series.
def load_files(folders):
    files = librosa.util.find_files(folders)
    laugh_audios = []
    for audio_name in files:
        y, sr = librosa.core.load(audio_name)
        laugh_audios.append(y)
    return laugh_audios

In [3]:
# Methods to compute the duration of all the file
def compute_duration(laugh_audios):
    audios_duration = []
    for i in laugh_audios:
        audios_duration.append(librosa.get_duration(i))
    return audios_duration

In [4]:
# Methods to compute average pitch (using estimate_tuning)
def compute_pitch(laugh_audios):
    audios_pitch = []
    for i in laugh_audios:
        audios_pitch.append((np.mean(librosa.feature.spectral_centroid(i))))
    return audios_pitch

In [15]:
def compute_energy(laugh_audios):
    audios_energy = []
    for i in laugh_audios:
        audios_energy.append([np.sqrt(np.dot(i, i))/len(i)][0])
    return audios_energy

In [6]:
# Method to scale the data to 0 - 10 so that we have a uniform standard for each attributes.
def nomalize_data(array):
    min_v, max_v = np.amin(array), np.amax(array)
    nom_array = []
    for i in array:
        if i == min_v:
            nom_array.append(0)
        else:
            nom_array.append(10*(i - min_v)/(max_v - min_v))
    return nom_array

In [7]:
def compute_tempo(laugh_audios):
    audios_tempo = []
    for i in laugh_audios:
        audios_tempo.append(librosa.beat.tempo(i))
    return audios_tempo

In [8]:
# Methods to compute features. Now: duration, avg pitches.
def compute_duration_and_pitch(folder_name):
    laughters = load_files(folder_name)
    durations = compute_duration(laughters) # Do not nomalized
    pitches = compute_pitch(laughters) # Do not nomalized
    data_points = []
    for i in range(len(laughters)):
        data_points.append([durations[i], pitches[i].round(decimals=3)])
    return data_points

In [9]:
# Methods to compute features. Now: tempo, avg pitches.
def compute_energy_and_duration(folder_name):
    laughters = load_files(folder_name)
    durations = compute_duration(laughters)
    energy = preprocessing.minmax_scale(compute_energy(laughters),feature_range = (1,8),axis =1)
    data_points = []
    for i in range(len(laughters)):
        data_points.append([durations[i], energy[i]])
    return data_points

In [16]:
JO_d_e = compute_energy_and_duration("JO_test")
print(JO_d_e)

[[1.2890702947845805, 1.0], [1.2900226757369615, 1.0], [0.76, 1.0], [0.5700226757369614, 1.0], [1.0, 1.0], [0.47002267573696144, 1.0], [0.9, 1.0], [1.3700226757369613, 1.0], [1.06, 1.0], [0.8700226757369615, 1.0], [0.9, 1.0], [1.0, 1.0], [3.1700226757369614, 1.0], [0.8700226757369615, 1.0], [1.5699773242630386, 1.0], [1.0500226757369615, 1.0], [1.84, 1.0], [2.0700226757369613, 1.0], [1.3300226757369615, 1.0]]


In [9]:
# Methods for applying K means cluster onto the final data_array. 
# Input: data_array. K (cluster numer). 
# Output: Print out each cluster sand their centers. Return labels.
def K_means(data_array, K):
    kmeans = KMeans(n_clusters = K).fit(data_array)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    clusters = [[] for i in range(K)] # Empty array to store the clusters.
    count = 0  # Count the index
    for ele in labels:
        clusters[ele].append(count)
        count += 1
    for i in range(K):
        print("Cluster", i ,":", clusters[i], " Centered at", centers[i])
    return labels

In [10]:
# Methods to draw the outcomes. Can change K values.
def make_graph(folder):
    data_points = np.array(compute_duration_and_pitch(folder)) # Change function here to get different features.
    labels = K_means(data_points, 3)
    print(data_points)
    #Plot the graph
    plt.scatter(data_points[labels == 0,0],data_points[labels == 0,1], c = 'b')
    plt.scatter(data_points[labels == 1,0],data_points[labels == 1,1], c = 'r')
    plt.scatter(data_points[labels == 2,0],data_points[labels == 2,1], c = 'g')
    plt.xlabel('duration of the laughter (scaled 0-10)')
    plt.ylabel('estimated tuning of the laughter (scaled 0-10)') # Change here depending on the features.
    plt.legend(('Cluster 0','Cluster 1','Cluster 2'))
    plt.show()

In [33]:
M_F = compute_duration_and_pitch("Male_Female")
print(M_F)

[[1.2890702947845805, 2773.967], [1.2900226757369615, 2155.264], [0.76, 1826.43], [0.5700226757369614, 1861.363], [1.0, 2172.306], [0.47002267573696144, 1918.463], [0.9, 2168.193], [1.3700226757369613, 2497.103], [1.06, 1859.309], [0.8700226757369615, 1686.663], [0.9, 1945.695], [1.0, 2633.5], [3.1700226757369614, 3007.214], [0.8700226757369615, 2303.767], [1.5699773242630386, 2336.101], [1.0500226757369615, 2125.836], [1.84, 2549.414], [2.0700226757369613, 2776.01], [1.3300226757369615, 2209.158]]


In [34]:
print(compute_duration_and_pitch("test_spec"))

[[0.9500226757369614, 1999.144], [0.76, 1826.43], [0.8500226757369614, 1997.063], [1.5100226757369615, 1648.398], [1.0500226757369615, 2125.836], [1.3900226757369614, 2100.247], [8.82, 2910.561]]


In [12]:
more = compute_duration_and_pitch("more")
print(more)

[[1.5300226757369615, 1268.233], [4.26, 1470.421], [5.227437641723356, 1685.525], [5.26, 1727.153], [2.1700226757369614, 1443.825], [0.84, 1765.336], [1.5500226757369615, 1461.732], [2.7099773242630385, 1711.693], [2.7700226757369615, 1919.905], [4.72, 1730.133], [2.8500226757369616, 2548.124]]
